In [1]:
source = "Recommendations";

In [2]:
using DataFrames
import CSV
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [3]:
ENV["COLUMNS"] = 999999;
ENV["LINES"] = 50;
recommend_related_series = false;

In [4]:
const anime = DataFrame(CSV.File("../../data/processed_data/anime.csv", ntasks = 1))
anime_to_uid = DataFrame(CSV.File("../../data/processed_data/anime_to_uid.csv"))
anime_to_uid = innerjoin(anime_to_uid, anime, on = "anime_id");

In [5]:
rating_df = DataFrame(
    "uid" => 0:num_items()-1,
    "ranking" => read_recommendee_alpha("BPR", "all").rating,
    "explicit" => read_recommendee_alpha("Explicit", "all").rating,
    "implicit" => read_recommendee_alpha("LinearImplicit", "all").rating,
    "error_explicit" => read_recommendee_alpha("ErrorExplicit", "all").rating,
    "error_implicit" => read_recommendee_alpha("ErrorImplicit", "all").rating,
    "baseline_explicit" =>
        read_recommendee_alpha("ExplicitUserItemBiases", "all").rating,
    "baseline_implicit" =>
        read_recommendee_alpha("NeuralImplicitUserItemBiases", "all").rating,
)
rating_df[:, "score"] = rating_df[:, "ranking"];

In [6]:
# evaluate our insample explicit predictions
df = get_recommendee_split(false)
rss = sum((df.rating - rating_df.explicit[df.item]) .^ 2)
tss = sum((df.rating .- mean(df.rating)) .^ 2)
@info "RMSE: $(sqrt(rss / length(df.rating)))"
@info "R2: $(1 - rss / tss)"

[ Info: 20220630 10:59:07 RMSE: 1.4020191
[ Info: 20220630 10:59:07 R2: 0.3389331


In [7]:
# [ Info: 20220629 11:36:49 RMSE: 1.4760647
# [ Info: 20220629 11:36:49 R2: 0.2672627

In [8]:
# evaluate our insample implicit predictions
df = get_recommendee_split(true)
p = rating_df.implicit[df.item]
y = df.rating / length(df.rating)
@info "Cross-entropy: $(sum(-y .* log.(p)))"

[ Info: 20220630 10:59:07 Cross-entropy: 7.4751287


In [9]:
# [ Info: 20220626 22:59:13 RMSE: 1.4526845
# [ Info: 20220626 22:59:13 R2: 0.29029125

In [10]:
# don't recommend shows that the user has already seen before
rating_df[:, "display"] .= true
seen_items = vcat(get_recommendee_split(true).item, get_recommendee_split(false).item)
rating_df.display[seen_items] .= false

# don't recommend shows related to shows they have seen before
if !recommend_related_series
    related_series = read_recommendee_alpha("ItemCFRelated", "all").rating .!= 0
    rating_df.display[related_series] .= false
end;

In [11]:
rec_df = innerjoin(anime_to_uid, rating_df, on = "uid");
keepcols = ["anime_id", "uid", "title", "genres", "medium"]
for x in names(rating_df)
    if x ∉ keepcols
        push!(keepcols, x)
    end
end
rec_df = rec_df[:, keepcols];

In [12]:
length(get_recommendee_split(false).item), length(get_recommendee_split(true).item)

(441, 443)

In [13]:
function pretty_display(df)
    sort(filter(x -> x.display, df), :score)
end;

In [14]:
filter(x -> x.medium == "tv", rec_df) |> pretty_display

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,display
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool
1,36259,10474,Pingu in the City,"['Comedy', 'Kids', 'Slice of Life']",tv,54.0,7.32275,6.53317e-5,1.07801,0.0401608,7.07446,5.37744e-5,54.0,1
2,38524,12753,Shingeki no Kyojin Season 3 Part 2,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,71.0,7.52367,0.00040463,0.828919,0.0185988,7.72584,0.00452048,71.0,1
3,32843,3602,Senki Zesshou Symphogear XV,"['Action', 'Idols (Female)', 'Music', 'Sci-Fi']",tv,79.0,7.49684,0.000146658,0.793819,0.0201014,6.98418,1.68961e-5,79.0,1
4,11061,7814,Hunter x Hunter (2011),"['Action', 'Adventure', 'Fantasy', 'Shounen']",tv,81.0,7.38618,0.000822997,1.03435,0.0217475,7.61737,0.00509626,81.0,1
5,16498,7644,Shingeki no Kyojin,"['Action', 'Drama', 'Gore', 'Military', 'Shounen', 'Survival']",tv,92.0,7.12432,0.00130991,0.83693,0.0212263,7.0864,0.00864467,92.0,1
6,32836,20123,Senki Zesshou Symphogear AXZ,"['Action', 'Idols (Female)', 'Music', 'Sci-Fi']",tv,100.0,7.32192,0.000144378,0.887158,0.0189914,6.34046,2.03424e-5,100.0,1
7,42941,9316,Uma Musume: Pretty Derby (TV) Season 2,"['Anthropomorphic', 'Sports']",tv,138.0,7.20405,0.000149508,0.776613,0.0174649,6.65906,2.75943e-5,138.0,1
8,2164,4538,Dennou Coil,"['Adventure', 'Comedy', 'Drama', 'Mystery', 'Sci-Fi']",tv,142.0,6.91257,0.00270957,0.673864,0.0277175,6.93308,7.60008e-5,142.0,1
9,21573,15434,Senki Zesshou Symphogear GX,"['Action', 'Idols (Female)', 'Music', 'Sci-Fi']",tv,158.0,7.11156,0.000140783,0.855747,0.0174807,6.1225,2.88984e-5,158.0,1


In [15]:
import Statistics: quantile

In [16]:
df = copy(rec_df)
df[:, "score"] = -(df[:, "explicit"] - df[:, "error_explicit"]);
filter(
    x -> x.explicit - x.baseline_explicit > 0.1,
    filter(x -> x.implicit > quantile(df.implicit, 0.01), df),
) |> pretty_display

,anime_id,uid,title,genres,medium,ranking,explicit,implicit,error_explicit,error_implicit,baseline_explicit,baseline_implicit,score,display
,Int64,Int64,String,String,String7,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Bool
1,48411,10158,Aria the Benedizione,"['Fantasy', 'Iyashikei', 'Sci-Fi', 'Shounen', 'Slice of Life']",movie,3719.0,7.78023,2.12903e-6,0.860498,0.00225001,6.92352,2.90412e-7,-6.91973,1
2,42984,8632,Gotcha!,"['Fantasy', 'Music']",music,63.0,7.67925,0.000108708,0.781197,0.0188144,7.45583,3.0276e-5,-6.89805,1
3,32843,3602,Senki Zesshou Symphogear XV,"['Action', 'Idols (Female)', 'Music', 'Sci-Fi']",tv,79.0,7.49684,0.000146658,0.793819,0.0201014,6.98418,1.68961e-5,-6.70303,1
4,10050,9463,Macross F Music Clip Shuu: Nyankuri,['Music'],music,524.0,7.41321,1.50553e-5,0.900859,0.0107849,6.19453,2.01177e-6,-6.51235,1
5,35472,7295,Yuuki Yuuna wa Yuushabu Shozoku 2,"['Comedy', 'Fantasy', 'Slice of Life']",movie,1510.0,7.49685,1.31532e-5,1.01703,0.010365,5.44444,1.15624e-6,-6.47982,1
6,33970,8707,Girls & Panzer: Saishuushou Part 1,"['Military', 'School']",movie,103.0,7.30118,0.00030287,0.835283,0.0193292,6.59781,3.65275e-5,-6.46589,1
7,35473,17744,Yuuki Yuuna wa Yuushabu Shozoku 3,"['Comedy', 'Fantasy', 'Slice of Life']",movie,1960.0,7.51696,1.25148e-5,1.06102,0.010365,5.482,1.02955e-6,-6.45594,1
8,32836,20123,Senki Zesshou Symphogear AXZ,"['Action', 'Idols (Female)', 'Music', 'Sci-Fi']",tv,100.0,7.32192,0.000144378,0.887158,0.0189914,6.34046,2.03424e-5,-6.43476,1
9,42941,9316,Uma Musume: Pretty Derby (TV) Season 2,"['Anthropomorphic', 'Sports']",tv,138.0,7.20405,0.000149508,0.776613,0.0174649,6.65906,2.75943e-5,-6.42744,1


In [17]:
# 	anime_id	uid	title	genres	medium	rating	alpha	std	p	nsfw
# Int64	Int64	String	String?	String7	Float32	Float32	Float32	Float32	String7
# 1	31988	11173	Hibike! Euphonium 2	['Drama', 'Music', 'School']	tv	7.38253	0.356698	0.792334	0.00168306	white
# 2	1065	967	Touch	['Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Sports']	tv	7.34929	0.467546	0.908755	0.00076882	white
# 3	27989	9666	Hibike! Euphonium	['Drama', 'Music', 'School']	tv	7.23239	0.571174	0.868754	0.00258483	white
# 4	1858	1694	Gakuen Utopia Manabi Straight!	['Comedy', 'School', 'Slice of Life']	tv	7.09079	1.20158	0.729596	0.00062308	white
# 5	25835	9373	Shirobako	['Comedy', 'Drama']	tv	7.14048	0.161561	0.805839	0.00261632	white
# 6	5941	4377	Cross Game	['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Sports']	tv	7.23503	0.154975	0.9538	0.00104641	white
# 7	39570	15596	High Score Girl II	['Comedy', 'Game', 'Romance', 'School', 'Seinen']	tv	7.01997	0.324915	0.878117	0.000592566	white
# 8	1530	1388	Kanon (2006)	['Drama', 'Romance', 'Slice of Life', 'Supernatural']	tv	7.16325	0.720318	1.03834	0.00227492	white
# 9	488	458	Ichigo Mashimaro	['Comedy', 'Slice of Life']	tv	6.9334	0.724677	0.852129	0.00104676	white
# 10	122	101	Full Moon wo Sagashite	['Comedy', 'Drama', 'Music', 'Romance', 'Shoujo', 'Supernatural']	tv	6.94031	0.727181	0.869193	0.000667557	white
# 11	21877	8534	High Score Girl	['Comedy', 'Game', 'Romance', 'School', 'Seinen']	tv	6.88657	0.471743	0.941942	0.000968524	white
# 12	38993	15192	Karakai Jouzu no Takagi-san 2	['Comedy', 'Romance', 'School', 'Shounen', 'Slice of Life']	tv	6.78872	0.179552	0.993493	0.000348786	white
# 13	532	499	Bishoujo Senshi Sailor Moon S	['Drama', 'Romance', 'Shoujo']	tv	6.63147	0.41036	0.84895	0.000437212	white
# 14	135	114	Hikaru no Go	['Comedy', 'Game', 'Shounen', 'Supernatural']	tv	6.74995	0.153899	1.02637	0.000676344	white
# 15	30727	10611	Saenai Heroine no Sodatekata ♭	['Comedy', 'Ecchi', 'Harem', 'Romance', 'School']	tv	6.59874	0.364116	0.876918	0.000742145	white
# 16	593	556	Mugen no Ryvius	['Drama', 'Mecha', 'Military', 'Psychological', 'Sci-Fi', 'Space']	tv	6.68717	0.304934	0.972565	0.00101426	white
# 17	34902	12650	Tsurezure Children	['Comedy', 'Romance', 'School', 'Shounen']	tv	6.51234	0.364974	0.807823	0.00102669	white
# 18	59	40	Chobits	['Comedy', 'Drama', 'Ecchi', 'Romance', 'Sci-Fi', 'Seinen']	tv	6.6161	0.883723	0.948174	0.00179486	white
# 19	38753	15022	Araburu Kisetsu no Otome-domo yo.	['Comedy', 'Drama', 'Romance', 'School', 'Shounen']	tv	6.52573	0.495667	0.864833	0.000905996	white
# 20	31771	11089	Amanchu!	['Comedy', 'School', 'Shounen', 'Slice of Life']	tv	6.58394	0.685901	0.926502	0.000584655	white
# 21	35860	13252	Karakai Jouzu no Takagi-san	['Comedy', 'Romance', 'School', 'Shounen', 'Slice of Life']	tv	6.59876	0.432538	0.977115	0.000852553	white
# 22	427	400	Kaleido Star	['Comedy', 'Drama', 'Fantasy', 'Shoujo', 'Sports']	tv	6.57368	0.344165	0.97538	0.000534053	white
# 23	40530	16120	Jaku-Chara Tomozaki-kun	['Drama', 'Romance', 'School']	tv	6.47727	0.923203	0.894883	0.000440828	white
# 24	874	789	Digimon Tamers	['Adventure', 'Comedy', 'Drama', 'Fantasy', 'Shounen']	tv	6.52593	0.48561	0.980111	0.000362535	white
# 25	35240	12889	Princess Principal	['Action', 'Historical', 'Mystery']	tv	6.5926	0.377174	1.06195	0.000912882	white
# 26	42361	16907	Ijiranaide, Nagatoro-san	['Comedy', 'Romance', 'Slice of Life']	tv	6.60228	0.83992	1.08517	0.000529261	white
# 27	18897	7968	Nisekoi	['Comedy', 'Harem', 'Romance', 'School', 'Shounen']	tv	6.31667	0.286932	0.808886	0.00141164	white
# 28	1486	1348	Kodomo no Omocha (TV)	['Comedy', 'Drama', 'Romance', 'Shoujo']	tv	6.56859	0.231561	1.06944	0.000533452	white
# 29	35639	13136	Just Because!	['Drama', 'Romance', 'School', 'Slice of Life']	tv	6.46695	0.763056	0.99794	0.000684533	white
# 30	1222	1114	Bokura ga Ita	['Drama', 'Romance', 'Shoujo', 'Slice of Life']	tv	6.70307	1.16039	1.23905	0.000494813	white
# 31	41619	16653	Munou na Nana	['Psychological', 'Shounen', 'Super Power', 'Supernatural', 'Suspense']	tv	6.41985	0.698886	0.958501	0.000440847	white
# 32	14131	7087	Girls & Panzer	['Action', 'Military', 'School', 'Sports']	tv	6.55237	0.586552	1.13453	0.00168731	white
# 33	552	517	Digimon Adventure	['Action', 'Adventure', 'Comedy', 'Fantasy', 'Kids']	tv	6.55298	0.365117	1.13609	0.000609451	white
# 34	40685	16205	Super Cub	['School', 'Slice of Life']	tv	6.35695	0.364119	0.944397	0.000580612	white
# 35	37890	14482	Oshi ga Budoukan Ittekuretara Shinu	['Comedy', 'Girls Love', 'Music', 'Seinen', 'Slice of Life']	tv	6.20357	0.206216	0.801772	0.000379987	white
# 36	34984	12704	Koi wa Ameagari no You ni	['Drama', 'Romance', 'Seinen', 'Slice of Life']	tv	6.56196	0.419388	1.16207	0.00116559	white
# 37	3958	3399	Kannagi	['Comedy', 'School', 'Shounen', 'Supernatural']	tv	6.24875	0.494657	0.866859	0.00120494	white
# 38	31859	11120	Hai to Gensou no Grimgar	['Action', 'Adventure', 'Drama', 'Fantasy']	tv	6.24894	0.166416	0.876223	0.000963634	white
# 39	13333	6942	Tari Tari	['Music', 'School', 'Slice of Life']	tv	6.24726	0.466783	0.878362	0.00116383	white
# 40	587	550	Hanbun no Tsuki ga Noboru Sora	['Comedy', 'Drama', 'Romance']	tv	6.30107	0.38663	0.960221	0.00128816	white
# 41	8861	5539	Yosuga no Sora: In Solitude, Where We Are Least Alone.	['Drama', 'Ecchi', 'Harem', 'Romance']	tv	7.46844	3.02749	2.15606	0.00104767	gray
# 42	2986	2716	Bamboo Blade	['Comedy', 'School', 'Seinen', 'Sports']	tv	6.17986	0.452029	0.868882	0.000749638	white
# 43	35756	13204	Comic Girls	['Comedy', 'Slice of Life']	tv	6.22388	0.350058	0.919891	0.000430328	white
# 44	1087	987	Kimagure Orange☆Road	['Comedy', 'Drama', 'Romance', 'School', 'Shounen', 'Slice of Life', 'Super Power']	tv	6.38255	0.190579	1.07896	0.000837759	white
# 45	12149	6691	AKB0048	['Music', 'Sci-Fi']	tv	6.26354	0.61283	0.960399	0.00053292	white
# 46	330	306	Midori no Hibi	['Comedy', 'Drama', 'Ecchi', 'Romance', 'Shounen']	tv	6.17071	0.533026	0.875367	0.00092522	white
# 47	46093	17949	Shiroi Suna no Aquatope	['Drama', 'Slice of Life']	tv	6.36729	0.377252	1.0729	0.000547827	white
# 48	5150	3975	Hatsukoi Limited.	['Comedy', 'Romance', 'School', 'Shounen']	tv	6.09554	0.397329	0.811726	0.000535075	white
# 49	740	677	Bishoujo Senshi Sailor Moon R	['Demons', 'Romance', 'Shoujo']	tv	6.29016	0.24701	1.00655	0.000616277	white
# 50	37982	14550	Domestic na Kanojo	['Drama', 'Romance', 'School', 